In [15]:
#웹 크롤링 라이브러리(beautifulsoup4) 설치 
!pip install beautifulsoup4
#selenium 설치
!pip install selenium
#webdriver-manager 설치
!pip install webdriver-manager

In [16]:
from bs4 import BeautifulSoup
import requests
import re
import time
import os
import sys
import urllib.request
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [8]:
# 데이터 저장 폴더 생성
import os
if not os.path.exists("crawled_data"):
    os.makedirs("crawled_data")

In [20]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

# Service 객체를 사용하여 드라이버 초기화
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(3)

In [22]:
import os
import json

# JSON 파일 생성 함수
def create_api_key_file(file_name):
    if not os.path.exists(file_name):
        default_keys = {
            "client_id": "your_client_id_here",
            "client_secret": "your_client_secret_here"
        }
        with open(file_name, 'w', encoding='utf-8') as f:
            json.dump(default_keys, f, ensure_ascii=False, indent=4)
        print(f"{file_name} 파일이 생성되었습니다. API 키를 입력하세요.")
    else:
        print(f"{file_name} 파일이 이미 존재합니다.")

# 파일 생성 실행
create_api_key_file("api_key.json")

api_key.json 파일이 이미 존재합니다.


In [35]:
import json

route = "./api_key.json"

# JSON 파일에서 API 키 읽기
with open(route, "r", encoding="utf-8") as file:
    api_keys = json.load(file)
try:
    client_id = api_keys["client_id"]
    client_secret = api_keys["client_secret"]
except KeyError:
    print("API 키가 JSON 파일에 없습니다. 확인 후 다시 시도하세요.")
    sys.exit(1)
else:
    print("API 키를 성공적으로 읽었습니다.")


API 키를 성공적으로 읽었습니다.


In [36]:
# selenium으로 검색 페이지 불러오기 #
naver_urls = []
postdate = []
titles = []

In [37]:
# 검색어 입력
keword = input("검색할 키워드를 입력해주세요:")
encText = urllib.parse.quote(keword)

In [41]:
# 검색을 끝낼 페이지 입력
end = input("\n크롤링을 끝낼 위치를 입력해주세요. (기본값:1, 최대값:100):")  
if end == "":
    end = 1
else:
    end = int(end)
print("\n 1 ~ ", end, "페이지 까지 크롤링을 진행 합니다")


 1 ~  1 페이지 까지 크롤링을 진행 합니다


In [42]:
# 한번에 가져올 페이지 입력
display = input("\n한번에 가져올 페이지 개수를 입력해주세요.(기본값:10, 최대값: 100):")
if display == "":
    display = 10
else:
    display = int(display)
print("\n한번에 가져올 페이지 : ", display, "페이지")


한번에 가져올 페이지 :  10 페이지


In [52]:
for start in range(end):
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText + "&start=" + str(start+1) + "&display=" + str(display+1) # JSON 결과
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        
        data = json.loads(response_body.decode('utf-8'))['items']
        for row in data:
            if('blog.naver' in row['link']):
                naver_urls.append(row['link'])
                postdate.append(row['postdate'])
                title = row['title']
                # html태그제거
                pattern1 = '<[^>]*>'
                title = re.sub(pattern=pattern1, repl='', string=title)
                titles.append(title)
        time.sleep(2)
    else:
        print("Error Code:" + rescode)

In [53]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

contents = []  # 블로그 내용을 저장할 리스트

try:
    for i, url in enumerate(naver_urls):
        print(f"블로그 URL ({i + 1}/{len(naver_urls)}): {url}")
        driver.get(url)
        time.sleep(3)  # 페이지 로딩 대기

        try:
            # iframe으로 전환
            iframe = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "mainFrame"))
            )
            driver.switch_to.frame(iframe)

            # iframe 내부의 HTML 가져오기
            source = driver.page_source
            html = BeautifulSoup(source, "html.parser")

            # 블로그 내용 추출
            content = html.select_one("div.se-main-container")  # 스마트에디터 블로그
            if content:
                content = content.get_text(strip=True)
            else:
                # 일반 블로그 처리
                content = html.select_one("div#postViewArea")
                content = content.get_text(strip=True) if content else "내용을 가져올 수 없습니다."

            contents.append(content)

        except Exception as e:
            print(f"오류 발생: {e}")
            contents.append("error")  # 오류 발생 시 'error' 추가

        finally:
            # iframe에서 기본 컨텍스트로 전환
            driver.switch_to.default_content()

    # 리스트 길이 동기화 확인
    while len(contents) < len(titles):
        contents.append("error")  # 누락된 데이터를 'error'로 채움

    # 데이터프레임 생성
    news_df = pd.DataFrame({'title': titles, 'content': contents, 'date': postdate})
    news_df.to_csv('blog.csv', index=False, encoding='utf-8-sig')

except Exception as e:
    print(f"전체 오류 발생: {e}")

블로그 URL (1/22): https://blog.naver.com/bylee19/223811086820
블로그 URL (2/22): https://blog.naver.com/ekt5587/223820500396
블로그 URL (3/22): https://blog.naver.com/haha_45/223832211719
블로그 URL (4/22): https://blog.naver.com/goldtop3321/223837014575
블로그 URL (5/22): https://blog.naver.com/yoonmh1230/223820062890
블로그 URL (6/22): https://blog.naver.com/dmsal217/223801117143
블로그 URL (7/22): https://blog.naver.com/gs_tnstngks/223844409469
블로그 URL (8/22): https://blog.naver.com/yaves0218/223821208727
블로그 URL (9/22): https://blog.naver.com/moon12_30/223842327111
블로그 URL (10/22): https://blog.naver.com/blvmasca/223782014104
블로그 URL (11/22): https://blog.naver.com/jiayoumi/223829059452
블로그 URL (12/22): https://blog.naver.com/bylee19/223811086820
블로그 URL (13/22): https://blog.naver.com/ekt5587/223820500396
블로그 URL (14/22): https://blog.naver.com/haha_45/223832211719
블로그 URL (15/22): https://blog.naver.com/goldtop3321/223837014575
블로그 URL (16/22): https://blog.naver.com/yoonmh1230/223820062890
블로그 URL 

In [56]:
news_df

,title,content,date
0,청주 인도커리 추천 성안길 맛집 '타즈마할 에베레스트 커리월드',​​안녕하세요! 부엉쓰 입니다 :)​​#청주맛집#청주맛집추천#성안길맛집#청주인도커리...,20250327
1,■ 청주 복대동 돈까스 맛집 - 카츠노하나 ■,안녕하세요 : )오늘은 청주 점심 맛집청주 지웰시티 돈까스 맛집인카츠노하나를 소개하...,20250403
2,청주 로컬 맛집 추천 부제 : 장원갑 칼국수 미나리 쌈 샤브,20250414​와이프가 칼국수를 좋아합니다.​그렇다 보니 저도 자주 먹는 음식중에...,20250414
3,청주 점심 맛집 백년가게 순대국전문점 부강옥,​청주 점심 맛집 부강옥에 다녀왔어요.언니집에 다니러 갔는데 점심 먹으러 가자며데리...,20250417
4,청주 고기집 밥집 지웰시티 맛집 숙성삼겹살 테이블더한,"글, 사진@윤씨청주 고기집 밥집 지웰시티 맛집 숙성삼겹살 테이블더한청주 고기집 밥집...",20250402
5,청주짬뽕맛집 분평짬뽕 솔직 후기:D (내부 주차 메뉴 총평),본 게시글은 소정의 원고료를 받아 작성되었습니다.청주짬뽕맛집 분평짬뽕 솔직 후기:D...,20250318
6,청주 가경동 터미널 맛집 샤브올데이 가격 무제한 뷔페 메뉴,​청주 가경동 터미널 맛집내 최애 음식 중 하나는 바로월남쌈 샤브샤브다.​그래서 집...,20250424
7,청주 가경동 맛집 가려다 아키아키 간 후기!,​​청주 가경동 맛집 가려다 아키아키 간 후기!청주 가경동 맛집 가려다 아키아키 간...,20250403
8,[청주 율량동] 우숙 - 율량동 한우 소고기 신상 맛집,​​청주 율량동 우숙 - 품질 좋은 상등육 한우를 배부르게 먹을 수 있는 율량동 소...,20250422
9,"청주 데이트맛집 파인다이닝 스시서울 오마카세 디너 후기, 추천",​​​청주 데이트맛집파인다이닝 스시서울 에서특별했던 일식 오마카세 경험 한 후기​#...,20250303
